# DEDUCE XSLX

Anonymize Excel sheets in Dutch using Deduce

For more info: https://github.com/vmenger/deduce or https://deduce.readthedocs.io/en/latest/

### 1. Install required dependencies

In [ ]:
!pip install deduce==3.0.3
!pip install pandas
!pip install openpyxl

### 2. Read Excel source file

In [ ]:
xlsx_file_name = "Dataset_mijnIBDcoach_NLP_tekstberichten_20240723_structuur.xlsx" # this file can be dropped in the left pane

import pandas as pd
df = pd.read_excel(xlsx_file_name, sheet_name=None) # read all sheets
sheet_names = df.keys()  # get all sheet names
print(sheet_names) # print all sheet names

### 3. Write an overview of the sheets-column names

In [ ]:

df_deduce_overview = pd.DataFrame(columns=['Sheet', 'Column', 'Anonymize'])
deduce_overview_file_name = "deduce_overview.xlsx"
sheet_list = []
column_list = []

for sheet in sheet_names:
    print(f"Sheet: {sheet}")
    df = pd.read_excel(xlsx_file_name, sheet_name=sheet) # read all sheets
    df.columns = df.iloc[0] # set the first row as column names
    columns = df.columns
    print(columns)
    for column in columns:
        sheet_list.append(sheet)
        column_list.append(column)
    
df_deduce_overview["Sheet"] = sheet_list
df_deduce_overview["Column"] = column_list
df_deduce_overview["Anonymize"] = False
df_deduce_overview.to_excel(deduce_overview_file_name, index=False)

### 4. Manually edit and save the overview "deduce_overview.xlsx"

### 5. Anonymize the sheets as indicated in the deduce_overview.xlsx file

In [ ]:
df_deduce_overview = pd.read_excel(deduce_overview_file_name)

from deduce import Deduce

deduce = Deduce()

def deduce_text(text):
    return deduce.deidentify(text).deidentified_text

for sheet in sheet_names:
    df = pd.read_excel(xlsx_file_name, sheet_name=sheet) # read all sheets
    df.columns = df.iloc[0] # set the first row as column names
    columns = df.columns

    for column in columns:
        # print(f"Sheet: {sheet} - Column: {column}")
        rows = df_deduce_overview.loc[(df_deduce_overview['Sheet'] == sheet) & (df_deduce_overview['Column'] == column)]
        if rows.empty:
            print(f"Warning! Sheet: {sheet} - Column: {column} not found in deduce_overview")
            continue
        if rows.iloc[0]["Anonymize"]:
            print(f"Anonymizing {sheet} - {column}")
            df[column] = df[column].apply(lambda x: deduce_text(x))
    
    df.to_excel(xlsx_file_name.replace(".xlsx", f"_{sheet}_anonymzed.xlsx"), index=False)